In [11]:
#find snps
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import allel
import bokeh
import iqplot
import numpy as np
import scipy.stats as st
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [13]:
root = "/Users/k23030440/"
snps_file = root + 'Jeffares copy/Jeffares SNPs/ERZ289255/Spombe.2013-01-02.filt3c.nr57-final.snps.anno-snpeff3.cleaned4.vcf'
indels_file = root + 'Jeffares copy/Jeffares SNPs/ERZ289254/Spombe.indels.unique.nr57.final.annotated.2014-01-23.cleaned4.vcf.gz'
final_results_screen = pd.read_csv(root + "S.-Pombe-MLPs - most recent/Bence folder/Bioneer screen/final_results_screen.csv")


In [15]:
#genome coordinates for genes
df = pd.read_csv(root + 'S.-Pombe-MLPs - most recent/external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.gff3', sep='\t', header=1)
#break last column into 2 at ;
df[['a','b']] = df["ID=SPAC1002.01;Name=mrx11"].str.split(';',expand=True)
#remove column ID=SPAC1002.01;Name=mrx11
df.drop(columns =["ID=SPAC1002.01;Name=mrx11"], inplace = True)
#remove ID= from column a and rename it to ID
df['ID'] = df['a'].str.replace('ID=', '')
#remove column a
df.drop(columns =['a','PomBase',".","..1"], inplace = True)
#rename b to Parent
df.rename(columns={'b':'Parent','gene':'object','I':'chromosome','1798347':"start",'1798835':"end","+":"strand"}, inplace=True)
#In parent remove anything before =
df['Parent'] = df['Parent'].str.replace('Parent=', '')
#or Name=
df['Parent'] = df['Parent'].str.replace('Name=', '')
#create an aempty column called detail
df['detail'] = ""
#split ID column at :, and if there is something after put it in detail column
df['detail'] = df['ID'].str.split(':', expand=True)[1]
#split ID column at :, and if there is something before put it in ID column
df['ID'] = df['ID'].str.split(':', expand=True)[0]#load in vcf file

In [16]:
df = df.loc[df["object"] == "gene"]
mediator = ["SPAC23H4.17c","SPBC12D12.06","SPAC688.08","SPAC589.02c","SPAC5D6.05","SPCC1450.05c"]

df = df.loc[df["ID"].isin(mediator) | df["ID"].isin(final_results_screen["strain"].values) | df["Parent"].isin(final_results_screen["strain"].values )]
df.reset_index(drop=True, inplace=True)
df.shape

(36, 8)

In [17]:
snps_data = allel.read_vcf(snps_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])
samples = snps_data['samples']
interesting_strains = samples
interesting_strains = ["JB914","JB871","JB1117", "JB872", "JB934", "JB759", "JB848","JB953","JB869"]

#finding the index of the interesting strains
interesting_strains_index = []
for i in range(len(samples)):
      if samples[i] in interesting_strains:
            interesting_strains_index.append(i)

calls= snps_data['calldata/GT'][:,:,0]
chromosomes = snps_data['variants/CHROM']
positions = snps_data['variants/POS']
alt = snps_data['variants/ALT']
ref= snps_data['variants/REF']


In [18]:
#iterate through positions
#results is a df with columns: alleles	n	sum_x	y_transpose_x	beta	standard_error	t_stat	p_value	type	allele1	allele2	chromosome	position
results = pd.DataFrame(columns=["strain","ID","chromosome","position","gene","type","allele1","allele2","p_value"])
#rough gene coordinates
#take mean of start and end
rough_gene_coordinates = (df["start"]+df["end"])/2


In [20]:

for i in range(len(positions)):
      #if positions[i] is not near to any of the rough_gene_coordinates  +-10000
      
      if np.isclose(positions[i],rough_gene_coordinates,atol=20000).any() == False:
            
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+200) and positions[i]>(df["start"][j]-200) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 2 strains with this value, then add the strain name to the detail column
                        if call_value == 0 or call_value == 1 or call_value == 2:
                              if sum(calls[i]==call_value) <= 5:
                                    
                                    #results = results.append({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"snp","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},ignore_index=True)
                                    results = pd.concat([results,pd.DataFrame({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"snp","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},index=[0])],ignore_index=True)


In [21]:
indels_data = allel.read_vcf(indels_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])

calls= indels_data['calldata/GT'][:,:,0]
chromosomes = indels_data['variants/CHROM']
positions = indels_data['variants/POS']
alt = indels_data['variants/ALT']
ref= indels_data['variants/REF']


In [23]:

for i in range(len(positions)):
   
      if np.isclose(positions[i],rough_gene_coordinates,atol=20000).any() == False:
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+200) and positions[i]>(df["start"][j]-200) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 3 strains with this value, then add the strain name to the detail column
                        if call_value == 0 or call_value == 1 or call_value == 2 or call_value == 3:
                              if sum(calls[i]==call_value) <= 3:
                                    
                                    results = pd.concat([results,pd.DataFrame({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"indel","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},index=[0])],ignore_index=True)


In [25]:
results

,strain,ID,chromosome,position,gene,type,allele1,allele2,p_value
0,JB872,SPAC1D4.06c,I,648673,csk1,snp,A,G,0.05
1,JB953,SPAC1D4.06c,I,648875,csk1,snp,C,T,0.05
2,JB872,SPAC1D4.06c,I,648962,csk1,snp,A,G,0.05
3,JB872,SPAC1D4.06c,I,649208,csk1,snp,T,C,0.05
4,JB872,SPAC1D4.06c,I,649624,csk1,snp,T,C,0.05
...,...,...,...,...,...,...,...,...,...
253,JB871,SPBC23G7.12c,II,2122791,rpt6,indel,T,TA,0.05
254,JB1117,SPBC12D12.06,II,2316851,srb11,indel,TA,T,0.05
255,JB759,SPBC12D12.06,II,2316851,srb11,indel,TA,T,0.05
256,JB872,SPBC405.07,II,3153756,rpl3602,indel,T,TTA,0.05


In [ ]:
#make csv file
#results.to_csv(root + "/MSci Bahler lab/S.-Pombe-MLPs - Github/Bence folder/Looking for variants in wild isolates/snp_hunting.csv", index=False)

In [ ]:
#analysis of nearby 100 SNPs to see whether JB914 and JB759 are different


In [73]:
interesting_strains_index

[10, 29, 44]

In [102]:
snps_data = allel.read_vcf(snps_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])
samples = snps_data['samples']
interesting_strains = samples
interesting_strains = ["JB914","JB759","JB22","JB869"]
#in df keep only row where Parent is srb11
df = df.loc[df["Parent"] == "srb11"]
#reset index
df.reset_index(drop=True, inplace=True)

#finding the index of the interesting strains
#create np array with same shape as interesting strains
interesting_strains_index =[]
#find index of each interesting strain in samples. If it is not in samples, put 0 in the array
for i in range(len(interesting_strains)):
      if interesting_strains[i] in samples:
            interesting_strains_index.append(np.where(samples==interesting_strains[i])[0][0])
      else:
            interesting_strains_index.append(0)


#if interesting_strains_index zero remove from interesting strains and interesting strains index
for i in range(len(interesting_strains_index)):
      if interesting_strains_index[i] == 0:
            interesting_strains.remove(interesting_strains[i])
            interesting_strains_index.remove(interesting_strains_index[i])
            break

In [103]:
interesting_strains_index

[44, 10, 29]

In [104]:
interesting_strains 

['JB914', 'JB22', 'JB869']

In [151]:


calls= snps_data['calldata/GT'][:,:,0]
chromosomes = snps_data['variants/CHROM']
positions = snps_data['variants/POS']
alt = snps_data['variants/ALT']
ref= snps_data['variants/REF']

#iterate through positions
#results is a df with columns: alleles	n	sum_x	y_transpose_x	beta	standard_error	t_stat	p_value	type	allele1	allele2	chromosome	position
results = pd.DataFrame(columns=["strain","ID","chromosome","position","gene","type","allele1","allele2","p_value"])
#rough gene coordinates
#take mean of start and end
rough_gene_coordinates = (df["start"]+df["end"])/2


for i in range(len(positions)):
      #if positions[i] is not near to any of the rough_gene_coordinates  +-10000
      
      if np.isclose(positions[i],rough_gene_coordinates,atol=100000).any() == False:
            
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+50000) and positions[i]>(df["start"][j]-100000) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 2 strains with this value, then add the strain name to the detail column
                        results = pd.concat([results,pd.DataFrame({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"snp","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},index=[0])],ignore_index=True)



indels_data = allel.read_vcf(indels_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])

calls= indels_data['calldata/GT'][:,:,0]
chromosomes = indels_data['variants/CHROM']
positions = indels_data['variants/POS']
alt = indels_data['variants/ALT']
ref= indels_data['variants/REF']


for i in range(len(positions)):
   
      if np.isclose(positions[i],rough_gene_coordinates,atol=100000).any() == False:
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+50000) and positions[i]>(df["start"][j]-100000) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 3 strains with this value, then add the strain name to the detail column
                        results = pd.concat([results,pd.DataFrame({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"indel","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},index=[0])],ignore_index=True)



In [152]:
results_final=results.copy()

In [153]:
#add column allele, which is allele2
results_final["allele"] = results_final["allele2"]
#where allele is empty, make it allele1
results_final.loc[results_final["allele"]=="","allele"] = results_final["allele1"]
results_final



,strain,ID,chromosome,position,gene,type,allele1,allele2,p_value,allele
0,JB914,SPBC12D12.06,II,2216902,srb11,snp,C,,0.05,C
1,JB22,SPBC12D12.06,II,2216902,srb11,snp,C,,0.05,C
2,JB869,SPBC12D12.06,II,2216902,srb11,snp,C,,0.05,C
3,JB914,SPBC12D12.06,II,2216992,srb11,snp,G,,0.05,G
4,JB22,SPBC12D12.06,II,2216992,srb11,snp,G,,0.05,G
...,...,...,...,...,...,...,...,...,...,...
8785,JB22,SPBC12D12.06,II,2366722,srb11,indel,TA,,0.05,TA
8786,JB869,SPBC12D12.06,II,2366722,srb11,indel,TA,,0.05,TA
8787,JB914,SPBC12D12.06,II,2366778,srb11,indel,AT,,0.05,AT
8788,JB22,SPBC12D12.06,II,2366778,srb11,indel,AT,,0.05,AT


In [154]:
#only keep strain position allele
results_final= results_final[["strain","position","allele"]]
results_final

,strain,position,allele
0,JB914,2216902,C
1,JB22,2216902,C
2,JB869,2216902,C
3,JB914,2216992,G
4,JB22,2216992,G
...,...,...,...
8785,JB22,2366722,TA
8786,JB869,2366722,TA
8787,JB914,2366778,AT
8788,JB22,2366778,AT


In [155]:
#remove duplicates for strain and position
results_final.drop_duplicates(subset=["strain","position"],inplace=True)
#reset index
results_final.reset_index(drop=True, inplace=True)

In [156]:
results_final

,strain,position,allele
0,JB914,2216902,C
1,JB22,2216902,C
2,JB869,2216902,C
3,JB914,2216992,G
4,JB22,2216992,G
...,...,...,...
8698,JB22,2366722,TA
8699,JB869,2366722,TA
8700,JB914,2366778,AT
8701,JB22,2366778,AT


In [157]:
#rearrange to make three columns for each strain
results_final = results_final.pivot(index='position', columns='strain', values='allele')
results_final

strain,JB22,JB869,JB914
position,,,
2216902,C,C,C
2216992,G,G,G
2217037,G,G,G
2217041,T,T,T
2217061,G,G,G
...,...,...,...
2367068,G,A,G
2367343,T,T,T
2367371,T,T,C


In [158]:
#how many of the rows show a difference between JB914 and JB759
diff= results_final.loc[(results_final["JB914"]!=results_final["JB22"])|(results_final["JB869"]!=results_final["JB22"])]
diff.head(50)

strain,JB22,JB869,JB914
position,,,
2219195,AT,AT,A
2219764,T,T,C
2221916,G,G,A
2222112,G,G,GA
2222671,T,T,C
2225261,A,A,G
2228418,T,C,C
2229030,A,G,A
2231797,C,C,A


In [ ]:
%load_ext watermark
%watermark


The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2023-11-03T18:58:41.209602+00:00

Python implementation: CPython
Python version       : 3.9.0
IPython version      : 7.31.1

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 21.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [ ]:
%watermark --iversions


numpy     : 1.23.4
matplotlib: 3.8.0
allel     : 1.3.5
pandas    : 1.4.4
bokeh     : 3.2.2
scipy     : 1.11.3
iqplot    : 0.3.3

